In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage.segmentation import watershed,find_boundaries
from skimage.feature import peak_local_max

import plotly.express as px

from skimage import io,util

import skimage.filters

from scipy.ndimage import gaussian_filter,morphology,label

from PIL import Image

from skimage.measure import regionprops

import plotly.graph_objects as go
import pandas as pd

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [3]:
#im_collection = io.imread_collection('0hrs/spheroids_231_cellsSF_hydrogels_3Ca/green/*.jpg')
im_collection = io.imread_collection('sample_B/dapi-blue/*.jpg')

im_3d = im_collection.concatenate()
data = util.img_as_float(im_3d[:,:,:,2])


#data=data[:,:900,:]
data=data[:,:500,:500]

# Rescale image data to range [0, 1]
data = np.clip(data, np.percentile(data, 0.5), np.percentile(data, 99.5))

#minimo=data.min()
#maximo=data.max()

#data = (data - data.min()) / (data.max() - data.min())

aux = gaussian_filter(data, sigma=1.0)
#aux=aux>0.1#skimage.filters.threshold_local(aux, 19, 'mean')

aux=aux>0.1
Dist = morphology.distance_transform_edt(aux)

local_maxi = peak_local_max(Dist, indices=False,min_distance=10,exclude_border=False)
neighbords=[[[1,1,1],
            [1,1,1],
            [1,1,1]],
            [[1,1,1],
            [1,1,1],
            [1,1,1]],
            [[1,1,1],
            [1,1,1],
            [1,1,1]]
           ]

markers = label(local_maxi,structure=neighbords)[0]
labels = watershed(-Dist, markers,mask=aux)

print(len(regionprops(labels)))

/home/hygor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning:

indices argument is deprecated and will be removed in version 0.20. To avoid this warning, please do not use the indices argument. Please see peak_local_max documentation for more details.



67


In [4]:
bound = find_boundaries(labels,mode="outer")

dim = np.zeros((len(data),500,500))
aa = np.stack((bound,dim, dim), axis=3)
bb = np.stack((dim,dim, data), axis=3)

In [ ]:
fig = px.imshow(200*aa+1000*bb, animation_frame=0, facet_col_wrap=2, binary_string=True)
fig.show()